# Proyecto Final - Modelos de Lenguaje
Este cuaderno reemplaza los scripts en Python planos y concentra en un solo lugar el flujo de trabajo del proyecto final.

## Cargar dependencias
Importamos las bibliotecas necesarias para manipular datos, entrenar y evaluar modelos de lenguaje.

In [ ]:
import json
from pathlib import Path
from typing import List, Dict

import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments


## Preparar datos
Las funciones siguientes cargan datos crudos desde archivos JSONL y los preparan para ser consumidos por un modelo de Transformers.

In [ ]:
def load_jsonl(path: Path) -> List[Dict]:
    """Carga un archivo JSONL y devuelve una lista de diccionarios."""
    with path.open("r", encoding="utf-8") as fh:
        return [json.loads(line) for line in fh]

def dataset_from_records(records: List[Dict]) -> Dataset:
    """Convierte la lista de ejemplos en un `datasets.Dataset`."""
    frame = pd.DataFrame.from_records(records)
    return Dataset.from_pandas(frame)

raw_records = load_jsonl(Path("data/train.jsonl"))
dataset = dataset_from_records(raw_records)
dataset


## Tokenización
Tokenizamos los ejemplos con el tokenizer adecuado al modelo base seleccionado.

In [ ]:
model_name = "dccuchile/bert-base-spanish-wwm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(batch: Dict[str, str]):
    return tokenizer(batch["text"], padding='max_length', truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset


## Entrenamiento
Se definen los argumentos y se lanza el proceso de fine-tuning del modelo de lenguaje.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
training_args = TrainingArguments(
    output_dir="models/spanish-bert-finetuned",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)

trainer.train()


## Evaluación y guardado
Calculamos métricas y guardamos los artefactos del modelo para su posterior reutilización.

In [ ]:
metrics = trainer.evaluate()
metrics

Path("models/spanish-bert-finetuned").mkdir(parents=True, exist_ok=True)
trainer.save_model()
tokenizer.save_pretrained("models/spanish-bert-finetuned")
